## Overview


One dimensional steady state groundwater flow in a confined aquifer can be modeled using the following PDE
$$
\begin{aligned}
-\frac{\partial}{\partial x}\left[ K(x) \frac{\partial h}{\partial x}\right] &= f(x)\\
h(0) &= 0\\
\left. \frac{\partial h}{\partial x}\right|_{x=1} &= 0,
\end{aligned}
$$
where $h(x)$ is the pressure (more generally the hydraulic head), $K(x)$ is a spatially distribution parameter called the hydraulic conductivity, and $f(x)$ is a source term.  For simplicity, we assume $x\in\Omega$, where $\Omega=[0,1]$.    While we will interpret this PDE through the lens of groundwater flow, the same equation arises in many disciplines, including heat conduction and electrostatics.  

<img src="DarcyFlow.png" style="float:right"></img>

The goal of this example is to illustrate how this flow equation can be implemented in a way that MUQ can understand.  In particular, we will define a child of MUQ's `ModPiece` class that approximately solves the flow equation with a finite element discretization.  The input of the `ModPiece` will be a vector defining `K(x)` and the output will be a vector of nodal $h(x)$ values.    In addition to solving the PDE, we will use adjoint techniques to provide efficient gradient and Hessian-action computations.

At the end of this example, the flow equation model is combined with a simple Gaussian density to imitate a Bayesian likelihood function.  The Hessian of this likelihood function is investigated by computing the Hessian's spectrum using MUQ's stochastic eigenvalue solver.


## Finite Element Formulation
Here we derive the finite element discretization as well as the adjoint equations for gradient and hessian operations.   *This section focuses exclusively on the details of the finite element discretization.  Nothing MUQ-specific is discussed until the `Implementation` section below.*  Readers familiar with finite element methods can skip ahead to that section with the knowledge that linear basis functions are used for representing $h(x)$, piecewise constant basis functions are used for $K(x)$, and all integrals are computed analytically.

#### Forward Model Discretization
To discretize the flow model PDE with a finite element method, we first need to consider the PDE in weak form.   Let $\mathcal{H}$ denote a Hilbert space of functions on $\Omega$ that satisfy the Dirichlet boundary condition $h(0)=0$.   For any function $p\in\mathcal{H}$, we have from the PDE that
$$
\int_\Omega p(x) \left[\frac{\partial}{\partial x}\left[ K(x) \frac{\partial h}{\partial x}\right] + f(x) \right] dx = 0
$$
Using integration by parts and applying the boundary conditions, we obtain the weak form of the flow equation, which is given by
$$
\int_\Omega K \frac{\partial p}{\partial x} \frac{\partial h}{\partial x} dx - \int_\Omega f p dx= 0.
$$
Our goal is to find the function $h$ such that this expression is satisfied for all functions $p$.  For this to be computationally feasible, we first need to represent the functions $h(x)$ and $p(x)$ through a finite number of basis functions.  Using a standard Galerkin discretization, we represent both $h$ and $p$ with the set of basis functiosn $\phi^1_i(x)$:
$$
\begin{aligned}
h(x) & = \sum_{i=1}^{N+1} h_i \phi^1_i(x)\\
p(x) & = \sum_{i=1}^{N+1} p_i \phi^1_i(x),
\end{aligned}
$$
where $h_i,p_i\in \mathbb{R}$ are weights on the basis function $\phi^1_i(x)$.  Here, we consider linear basis functions (also known as "hat" functions) given by 
$$
\phi^1_i(x) = \left\{ \begin{array}{ll} \frac{x-x_{i-1}}{x_i-x_{i-1}} & x\in[x_{i-1},x_i]\\ \frac{x_{i+1}-x}{x_{i+1}-x_i} & x\in(x_i,x_{i+1}] \\ 0 & \text{otherwise} \end{array} \right.,
$$
where $[x_0,x_1,\ldots,x_{N+1}]$ is an increasing sequence of node locations splitting the domain $\Omega$ into $N$ cells.  We will further assume uniform node spacing is constant so that $x_{i+1}-x_i = \delta x$ for all $i$.

A similar approach is used to discretize $K(x)$ and $f(x)$.  In particular, we let 
$$
\begin{aligned}
K(x) &= \sum_{i=1}^N K_i \phi^0_i(x)\\
f(x) &= \sum_{i=1}^N f_i \phi^0_i(x),
\end{aligned}
$$
but consider piecewise constant basis functions 
$$
\phi^0_i(x) = \left\{\begin{array}{ll} 1 & x\in[x_{i},x_{i+1})\\ 0 & \text{otherwise} \end{array}\right.
$$
so that $K(x)$ and $f(x)$ are piecewise constant.  With these discretizations in hand, the weak form becomes
$$
\begin{aligned}
\int_\Omega K \frac{\partial p}{\partial x} \frac{\partial h}{\partial x} dx - \int_\Omega f p dx & = \int_\Omega \left(\sum_{k=1}^N K_k \phi_k^0(x) \right) \left(\sum_{i=1}^{N+1}p_i \frac{\partial \phi^1_i}{\partial x}\right) \left(\sum_{j=1}^{N+1}h_j \frac{\partial \phi^1_j}{\partial x}\right) dx - \int_\Omega \left(\sum_{k=1}^N f_k \phi_k^0(x) \right) \left(\sum_{i=1}^{N+1}p_i \phi^1_i \right) dx.
\end{aligned}
$$
For this quantity to be zero for all functions $p\in\text{span}(\{\phi_1^1,\ldots,\phi_{N+1}^1\})$, we require that 
$$
\begin{aligned}
& \int_\Omega \frac{\partial \phi^1_i}{\partial x} \left(\sum_{k=1}^N K_k \phi_k^0(x) \right)  \left(\sum_{j=1}^{N+1}h_j \frac{\partial \phi^1_j}{\partial x}\right) dx  - \int_\Omega \phi_i^1 \left(\sum_{k=1}^N f_k \phi_k^0(x) \right) dx &= 0\\
\Rightarrow & \sum_{j=1}^{N+1}\sum_{k=1}^N \left[ h_j K_k \int_\Omega \phi_k^0(x) \frac{\partial \phi^1_i}{\partial x} \frac{\partial \phi^1_j}{\partial x} dx - f_k \int_\Omega \phi_i^1(x)\phi_k^0(x) dx \right] & = 0
\end{aligned}
$$
for all $i$.  Notice that this defines a linear system of equations $\mathbf{K} \mathbf{h} = \mathbf{f}$, where $\mathbf{h}=[h_1,\ldots,h_{N+1}]$ is a vector of coefficients, $\mathbf{K}$ is a matrix with components given by 
$$
\mathbf{K}_{ij} = \sum_{k=1}^N K_k \int_{x_k}^{x_{k+1}} \frac{\partial \phi^1_i}{\partial x} \frac{\partial \phi^1_j}{\partial x} dx,
$$
and 
$$
\mathbf{f}_i = \sum_{k=1}^N f_k \int_{x_k}^{x_{k+1}} \phi_i^1(x) dx.
$$

To approximately solve the flow equation, we therefore need to construct the stiffness matrix $\mathbf{K}$ and the right hand side vector $\mathbf{f}$ before solving $\mathbf{u} = \mathbf{K}^{-1}\mathbf{f}$ to obtain the coefficients $h_i$.



#### Adjoint-based Gradient Computation
Many sampling, optimization, and dimension reduction algorithms require derivative information.   Here we consider adjoint strategies for efficient computing gradients of functionals $J(h)$ with respect to the conductivity field $K(x)$.  A natural way to do this is to consider the following constrained optimization problem
$$
\begin{aligned}
\min_{h,k}\quad & J(h)\\
\text{s.t.}\quad & \frac{\partial}{\partial x}\left[ k(x) \frac{\partial h}{\partial x}\right] + f(x) = 0.
\end{aligned}
$$ 
Using a Lagrange multiplier $p(x)$, we can form the Lagrangian $\mathcal{L}(h,k,p)$ associated with this optimization problem and solve for optimality conditions. The Lagrangian is given by
$$
\mathcal{L}(h,k,p) = J(h) - \int_\Omega p \frac{\partial}{\partial x}\left[ k \frac{\partial h}{\partial x}\right] dx - \int_\Omega p f dx.
$$ 
Note that the explicit dependence of $h$, $k$, and $p$ on $x$ has been dropped for clarity.    Using integration by parts, we can "move" the derivative on $k \frac{\partial h}{\partial x}$ to a derivative on $p$, resulting in 
$$
\mathcal{L}(h,k,p) = J(h) - \left.\left(p k \frac{\partial h}{\partial x}\right)\right|_{0}^{1} + \int_\Omega k \frac{\partial p}{\partial x} \frac{\partial h}{\partial x} dx - \int_\Omega p f dx.
$$
Recall that $p(0)=0$ because of the Dirichlet condition at $x=0$.  Combined with the boundary condition $\partial h / \partial dx = 0$ at $x=1$, the Lagrangian can be simplified to 
$$
\mathcal{L}(h,k,p) = J(h) + \int_\Omega k \frac{\partial p}{\partial x} \frac{\partial h}{\partial x} dx - \int_\Omega p f dx.
$$

We now consider the first variations of the Lagrangian with respect to each input.   Recall that the first variation of a functional $G(m)$ is a function $G_m(m)(\tilde{m})$ representing the directional derivative of the functional $G$ in the direction $\tilde{m}$ and is defined as 
$$
G_m(m)(\tilde{m}) = \left. \frac{\partial}{\partial \epsilon} G(m+\epsilon\tilde{m}) \right|_{\epsilon=0},
$$
where $\epsilon\in\mathbb{R}$ is a scalar step size.

Using this definition, the first variation of $\mathcal{L}$ with respect to $h$ is given by
$$
\begin{aligned}
\mathcal{L}_h(h,k,p)(\tilde{h}) &= \left. \frac{\partial}{\partial \epsilon} \mathcal{L}(h+\epsilon \tilde{h},k,p) \right|_{\epsilon=0}\\
&= \left. \frac{\partial}{\partial \epsilon}\left[J(h+\epsilon \tilde{h}) +\int_\Omega k \frac{\partial p}{\partial x} \frac{\partial (h+\epsilon\tilde{h})}{\partial x} dx - \int_\Omega p f dx\right] \right|_{\epsilon=0}\\
&= J_h(h)(\tilde{h}) + \int_\Omega k \frac{\partial p}{\partial x}\frac{\partial \tilde{h}}{\partial x} dx.
\end{aligned}
$$

Similarly, the first variation with respect $p$ is 
$$
\begin{aligned}
\mathcal{L}_p(h,k,p)(\tilde{p}) &= \left. \frac{\partial}{\partial \epsilon} \mathcal{L}(h,k,p+\epsilon \tilde{p}) \right|_{\epsilon=0}\\
&= \left. \frac{\partial}{\partial \epsilon}\left[J(h) + \int_\Omega k \frac{\partial (p+\epsilon \tilde{p})}{\partial x} \frac{\partial h}{\partial x} dx - \int_\Omega (p+\epsilon \tilde{p}) f dx\right] \right|_{\epsilon=0}\\
&= \int_\Omega k \frac{\partial \tilde{p}}{\partial x}\frac{\partial h}{\partial x} dx - \int_\Omega \tilde{p} f dx.
\end{aligned}
$$

And finally, the first variation with respect to $k$ is given by
$$
\begin{aligned}
\mathcal{L}_k(h,k,p)(\tilde{k}) &= \left. \frac{\partial}{\partial \epsilon} \mathcal{L}(h,k+\epsilon \tilde{k},p) \right|_{\epsilon=0}\\
&= \left. \frac{\partial}{\partial \epsilon} \left[ J(h) + \int_\Omega (k+\epsilon\tilde{k}) \frac{\partial p}{\partial x} \frac{\partial h}{\partial x} dx - \int_\Omega p f dx\right] \right|_{\epsilon=0}\\
&=\int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial p}{\partial x} dx
\end{aligned}
$$

The solution of the optimization problem is given by the functions $(h,k,p)$ where all of these first variations are zero for all admissable directions $\tilde{h}$, $\tilde{p}$, and $\tilde{k}$.  Solving  $\mathcal{L}_p(h,k,p)(\tilde{p})=0$ for all $\tilde{p}$ yields the weak form of the original elliptic PDE -- the "forward equation."  Solving $\mathcal{L}_h(h,k,p)(\tilde{h})$ for all $\tilde{h}$ results in the weak form of the adjoint equation.   Finally, the constraint $\mathcal{L}_k(h,k,p)(\tilde{k})=0$ provides a definition of the gradient using the solutions of the forward and adjoint equations.   

To see that $\mathcal{L}_k(h,k,p)(\tilde{k})=0$ defines a gradient, consider the relationship between the gradient and directional derivative of a function $g(y)$ mapping $\mathbb{R}^N\rightarrow\mathbb{R}$.   The derivative of $g$ in a unit direction $\tilde{y}$ is given by the inner product of the gradient and the direction: $\tilde{y}\cdot \nabla_y g$.  Comparing this to the first variation $\mathcal{L}_k(h,k,p)(\tilde{k})$, we see that the integral $\int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial p}{\partial x} dx$ is just an inner product of the direction $\tilde{k}$ with $\nabla_kJ = \frac{\partial h}{\partial x} \frac{\partial p}{\partial x}$.  $\nabla_kJ$ is therefore the functional gradient of $J$ with respect to the conductivity $K$.

Notice that the adjoint equation has the same form as the forward model but with a different right hand side -- the flow equation is self-adjoint.   This means that we can use the same finite element discretization described above to solve the adjoint equation.  In particular, we can solve the linear system $\mathbf{K} \mathcal{p} = \nabla_h J$, where $\nabla_h J$ is a vector containing the gradient of $J$ with respect to the coefficients $\mathbf{h}_i$ defining the solution $h(x)$.  The solution of this linear system can then be used to obtain the derivative of $J$ with respect to each coefficient $K_i$ defining the conductivity field.  In particular,
$$
\frac{\partial J}{\partial K_i} = \frac{ \mathbf{h}_{i+1}-\mathbf{h}_{i}}{x_{i+1}-x_i} \frac{\mathbf{p}_{i+1} - \mathbf{p}_i}{x_{i+1} - x_i}.
$$

Note that obtaining the gradient with this approach only requires one extra linear solve to obtain the adjoint variable $\mathbf{p}$ and then the gradient $\nabla_K J$.  This is in stark contrast to a finite difference approximation that would require $N$ additional solves to obtain each $\frac{\partial J}{\partial K_i}$.  

#### Adjoint-based Hessian actions
Second derivative information can also be exploited by many different UQ, optimization, and dimension reduction algorithms.  Newton methods in optimization are a classic example of this.  Above we used an adjoint approach to efficient construct the gradient.  Here we use a similar approach to efficiently evaluate the action of the Hessian operator $\nabla_{KK} J$ on a function $\hat{K}$.   After discretizing, this is equivalent to multiplying the Hessian matrix times a vector.

Recall that $\mathcal{L}_k(h,k,p)(\tilde{k})$ is a scalar quantity that, when $\mathcal{L}_p(h,k,p)(\tilde{p})=0$ for all $\tilde{p}$ and $\mathcal{L}_h(h,k,p)(\tilde{h})=0$ for all $\tilde{h}$, represents the directional derivative of the Lagrangian in direction $\tilde{k}$.   To characterize the Hessian, we now want to consider the change in this scalar quantity when $k$ is perturbed in another direction.  In an abstract sense, this is identical to our adjoint approach for the gradient, however, instead of considering the scalar functional $J(h)$, we are considering the scalar functional $\mathcal{L}_k(h,k,p)(\tilde{k})$.

To characterize a second directional derivative, we can form another (meta-)Lagrangian using $\mathcal{L}_k(h,k,p)(\tilde{k})$ and the constraints $\mathcal{L}_p(h,k,p)(\tilde{p})=0$ and $\mathcal{L}_h(h,k,p)(\tilde{h})=0$.  The strong form of these constraints is given by
$$
\begin{aligned}
-\frac{\partial}{\partial x}\left[ k(x) \frac{\partial h}{\partial x}\right] &= f(x)\\
h(0) &= 0\\
\left. \frac{\partial h}{\partial x}\right|_{x=1} &= 0,
\end{aligned}
$$
and 
$$
\begin{aligned}
-\frac{\partial}{\partial x}\left[ k(x) \frac{\partial p}{\partial x}\right] &= D_hJ(x)\\
p(0) &= 0\\
\left. \frac{\partial p}{\partial x} \right|_{x=1} &= 0,
\end{aligned}
$$

In addition to introducing two additional Lagrange multipliers $\hat{p}$ and $\hat{h}$, we will also introduce a new function $s=\nabla_hJ(h)$ to represent the sensitivity of the objective functional $J(h)$ with respect to $h$.  Notice that we will not directly consider second variations of $J$.  Instead, we independently consider the action of Hessian on functions $\bar{k}$ and $\bar{s}$, which is what MUQ will need to propagate second derivative information through a composition of model components.

Using $\hat{p}$, $\hat{h}$, and $s$, the meta-Lagrangian $\mathcal{L}^H$ takes the form
$$
\begin{aligned}
\mathcal{L}^H(h,k,p,\hat{h},\hat{p}, s; \tilde{k}) &= \int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial p}{\partial x} dx - \int_\Omega \hat{p} \left( \frac{\partial}{\partial x}\left[ k \frac{\partial h}{\partial x}\right] + f\right) dx - \int_\Omega \hat{h}\left(\frac{\partial}{\partial x}\left[ k \frac{\partial p}{\partial x}\right] + s \right) dx \\
&=  \int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial p}{\partial x} dx + \int_\Omega k \frac{\partial \hat{p}}{\partial x}\frac{\partial h}{\partial x} dx - \int_\Omega \hat{p} f dx + \int_\Omega k \frac{\partial \hat{h}}{\partial x}\frac{\partial p}{\partial x} dx - \int_\Omega \hat{h} s dx
\end{aligned}
$$
Placing the $\tilde{k}$ after the semicolon in the arguments to  $\mathcal{L}^H$ is meant to indicate that the direction $\tilde{k}$ is a parameter defining the first directional derivative and is not an argument that needs to be considered when taking first variations.


The first variation with respect to $p$ is given by
$$
\begin{aligned}
\mathcal{L}_{p}^H(h,k,p,\hat{h},\hat{p}, s; \tilde{k})(\bar{p}) & = \left. \frac{\partial}{\partial \epsilon} \mathcal{L}^H(h,k,p+\epsilon\bar{p},\hat{h},\hat{p}, s; \tilde{k}) \right|_{\epsilon=0}\\
& =  \left. \frac{\partial}{\partial \epsilon} \left( \int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial (p+\epsilon\bar{p})}{\partial x} dx + \int_\Omega k \frac{\partial \hat{p}}{\partial x}\frac{\partial h}{\partial x} dx - \int_\Omega \hat{p} f dx + \int_\Omega k \frac{\partial \hat{h}}{\partial x}\frac{\partial (p+\epsilon\bar{p})}{\partial x} dx \right)\right|_{\epsilon=0}\\
& = \int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial \bar{p}}{\partial x} dx + \int_\Omega k \frac{\partial \hat{h}}{\partial x}\frac{\partial \bar{p}}{\partial x} dx 
\end{aligned}
$$
This expression defines what is commonly called the *incremental forward problem.*

The first variation of this meta-Lagrangian with respect to $h$ is given by.
$$
\begin{aligned}
\mathcal{L}_{h}^H(h,k,p,\hat{h},\hat{p}, s; \tilde{k})(\bar{h}) & = \left. \frac{\partial}{\partial \epsilon} \mathcal{L}^H(h+\epsilon\bar{h},k,p,\hat{h},\hat{p}, s; \tilde{k}) \right|_{\epsilon=0}\\
&= \left. \frac{\partial}{\partial \epsilon} \left( \int_\Omega \tilde{k}\frac{\partial (h+\epsilon\bar{h})}{\partial x} \frac{\partial p}{\partial x} dx + \int_\Omega k \frac{\partial \hat{p}}{\partial x}\frac{\partial (h+\epsilon\bar{h})}{\partial x} dx - \int_\Omega \hat{p} f dx + \int_\Omega k \frac{\partial \hat{h}}{\partial x}\frac{\partial p}{\partial x} dx - \int_\Omega \hat{h}\, s dx \right)\right|_{\epsilon=0} \\
&= \int_\Omega \tilde{k}\frac{\partial \bar{h}}{\partial x} \frac{\partial p}{\partial x} dx + \int_\Omega k \frac{\partial \hat{p}}{\partial x}\frac{\partial \bar{h}}{\partial x} dx.
\end{aligned}
$$
This expression defines the *incremental adjoint problem.*

Finally, the first variation with respect to $k$ is
$$
\begin{aligned}
\mathcal{L}_{k}^H(h,k,p,\hat{h}, \hat{p}, s; \tilde{k})(\bar{k}) &= \left. \frac{\partial}{\partial \epsilon} \mathcal{L}^H(h,k+\epsilon \bar{k},p,\hat{h},\hat{p},s; \tilde{k}) \right|_{\epsilon=0}\\
&= \left. \frac{\partial}{\partial \epsilon}\left( \int_\Omega \tilde{k}\frac{\partial h}{\partial x} \frac{\partial p}{\partial x} dx + \int_\Omega (k+\epsilon \bar{k}) \frac{\partial \hat{p}}{\partial x}\frac{\partial h}{\partial x} dx - \int_\Omega \hat{p} f dx + \int_\Omega (k+\epsilon\bar{k}) \frac{\partial \hat{h}}{\partial x}\frac{\partial p}{\partial x} dx - \int_\Omega \hat{h} D_hJ dx \right) \right|_{\epsilon=0} \\
&= \int_\Omega \bar{k} \frac{\partial \hat{p}}{\partial x}\frac{\partial h}{\partial x} dx +\int_\Omega\bar{k} \frac{\partial \hat{h}}{\partial x} \frac{\partial p}{\partial x} dx.
\end{aligned}
$$
Just like the first variation with respect to $K$ allowed us to compute the gradient.  This expression allows us to compute the Hessian action.    In particular, the action of the Hessian $\nabla_{KK}J$ on the function $\tilde{K}$ is given by 
$$
(\nabla_{KK}J ) \tilde{K} = \frac{\partial \hat{p}}{\partial x}\frac{\partial h}{\partial x} + \frac{\partial \hat{h}}{\partial x} \frac{\partial p}{\partial x}.
$$
Note that $\tilde{k}$ is implicitly contained in the right hand side of this expression through $\hat{p}$ and $\hat{h}$.

## Implementation
MUQ implements components of models through the `ModPiece`.  Readers that are not familiar with the `ModPiece` class should consult the [model compoents tutorial](https://mituq.bitbucket.io/source/_site/latest/group__modpieces.html) before proceeding. 

To define the adjoint-enabled flow equation solver derived above, we need to override 3 methods in the `ModPiece` base class: the `EvaluateImpl` function, which will solve $\mathbf{K}\mathbf{h}=\mathbf{f}$, the `GradientImpl` function, which will solve $\mathbf{K}p=\mathbf{s}$ and compute the gradient, and finally, the `ApplyHessianImpl` function, which will solve the incremtnal forward and adjoint problems for $\hat{\mathbf{p}}$ and $\hat{\mathbf{h}}$ before returning the Hessian action.  These functions are all contained in the `FlowEquation` class below.

Notice that all of these function require constructing the stiffness matrix $\mathbf{K}$.   To avoid duplicitous code, the `FlowEquation` class also contains a `BuildStiffness` function that accepts a numpy vector of hydraulic conductivities and returns a scipy sparse matrix containing $\mathbf{K}$.

To implement the boundary Dirichlet boundary condition $h(x=0)=0$, we ensure that the first component $\mathbf{f}_0 = 0$ and that  $\mathbf{K}_{0,0}=c$ for some large value $c$ (e.g., $10^{10}$).   This is a common "trick" for enforcing Dirichlet boundary conditions while ensuring the stiffness matrix $\mathbf{K}$ remains symmetric. 

### Imports
In addition to the MUQ modeling module, we'll need numpy and scipy for sparse linear algebra.  Matplotlib will also be used for plotting.

In [1]:
import muq.Modeling as mm

import numpy as np

import scipy.sparse as sp
import scipy.sparse.linalg as spla

import matplotlib.pyplot as plt


### Class Definition

In [2]:
class FlowEquation(mm.PyModPiece):
    """
    This class solves the 1D elliptic PDE of the form
    $$
    -\frac{\partial}{\partial x}\cdot(K(x) \frac{\partial h}{\partial x}) = f(x).
    $$
    over $x\in[0,1]$ with boundary conditions $h(0)=0$ and
    $\partial h/\partial x =0$ at $x=1$.  This equation is a basic model of steady
    state fluid flow in a porous media, where $h(x)$ is the hydraulic head, $K(x)$
    is the hydraulic conductivity, and $f(x)$ is the recharge.

    This ModPiece uses linear finite elements on a uniform grid. There is a single input,
    the conductivity $K(x)$, which is represented as piecewise constant within each
    of the $N$ cells.   There is a single output of this ModPiece: the head $h(x)$ at the
    $N+1$ nodes in the discretization.
    
    INPUTS:
        sourceTerm (np.array) : A numpy array containing the value of the source term $f(x)$ in each grid cell.

    """

    def __init__(self, sourceTerm):
        super(FlowEquation,self).__init__([sourceTerm.shape[0]],   # inputSizes  (one for each cell)
                                               [sourceTerm.shape[0]+1]) # outputSizes (one for each node)
        
        self.numCells = sourceTerm.shape[0]
        self.numNodes = self.numCells+1
        self.source = sourceTerm

        self.xs = np.linspace(0,1,self.numNodes) # Assumes domain is [0,1]
        self.dx = self.xs[1]-self.xs[0] 
        
        self.rhs = self.BuildRhs(self.source)
        
        #self.massMat = self.BuildMassMatrix()
        
        
    def EvaluateImpl(self, inputs):
        """ Constructs the stiffness matrix and solves the resulting linear system.

            INPUTS:
                inputs: A list of vector-valued inputs.  Our model only has a single input,
                        so this list will also have one entry containing a vector of
                        conductivity values for each cell.

            RETURNS:
                This function returns nothing.  It stores the result in the private
                ModPiece::outputs list that is then returned by the `Evaluate` function.
        """

        condVals = inputs[0]

        # Build the stiffness matrix and right hand side
        K = self.BuildStiffness(condVals)

        # Solve the sparse linear system
        sol = spla.spsolve(K,self.rhs)

        # Set the output list using the solution
        self.outputs = [sol]

    def GradientImpl(self, outWrt, inWrt, inputs, sensitivity):
        """ This function computes one step of the chain rule to compute the gradient 
            $\nabla_k J$ of $J$ with respect to the input parameters.  In addition 
            to the model parameter $k$, this function also requires the sensitivity of 
            $J$ with respect to the flow equation output, i.e. $\nabla_h J$.
        
            The gradient with respect to the conductivity field is computed by solving
            the forward model, solving the adjoint system, and then combining the results to
            obtain the gradient.

            INPUTS:
                outWrt: For a model with multiple outputs, this would be the index
                        of the output list that corresponds to the sensitivity vector.
                        Since this ModPiece only has one output, the outWrt argument
                        is not used in the GradientImpl function.

                inWrt: Specifies the index of the input for which we want to compute
                       the gradient.  For inWrt==0, then the gradient with respect
                       to the conductivity is returned.  Since this ModPiece only has one 
                       input, 0 is the only valid value for inWrt.

                inputs: Just like the EvalauteImpl function, this is a list of vector-valued inputs.

                sensitivity: A vector containing the gradient of an arbitrary objective $J$
                             with respect to the output of this ModPiece.

            RETURNS:
                This function returns nothing.  It stores the result in the private
                ModPiece::gradient variable that is then returned by the `Gradient` function.

        """
        condVals = inputs[0]

        # Construct the adjoint system
        K = self.BuildStiffness(condVals)
        adjRhs = self.BuildAdjointRhs(sensitivity)
        
        # Solve the adjoint system
        sol = spla.spsolve(K,self.rhs)
        adjSol = spla.spsolve(K.T,adjRhs)

        # Compute the gradient from the adjoint solution
        dhdx = (sol[1:] - sol[:-1])/self.dx   # derivative of forward solution, which is constant in each cell
        dpdx = (adjSol[1:] - adjSol[:-1])/self.dx # derivative of adjoint solution, which is constant in each cell
        
        self.gradient = self.dx * dhdx * dpdx 
        
    
    def ApplyHessianImpl(self, outWrt, inWrt1, inWrt2, inputs, sensitivity, vec):
        """  Computes the action of Hessian on a vector.
        
             To understand this function, it is useful to interpret the `GradientImpl` function 
             as a mapping $g(k,s)$ that accepts parameters $k$ as well as a sensitivity vector $s$ 
             and returns the gradient of $J$.  The Hessian matrix can be thought of as the Jacobian 
             of this mapping $g(k,s)$.   This function therefore applies either the Jacobian with 
             respect to $k$, $D_k g$, or the Jacobian with respect to $s$, $D_s g$, 
             to a vector $v$.   

            INPUTS:
                outWrt: For a model with multiple outputs, this would be the index
                        of the output list that corresponds to the sensitivity vector.
                        Since this ModPiece only has one output, the outWrt argument
                        is not used in the ApplyHessianImpl function.

                inWrt1: Specifies the index of the first input that we want to differentiate 
                        with respect to.  Since there is only one input to this ModPiece,
                        inWrt1 will also be 0.
                
                inWrt2: Specifies the index of the second input that we want to differentiate 
                        with respect to.  This second wrt argument can also be used to specify 
                        derivatives with respect to the sensitivity input $s$.  For this example,
                        if `inWrt2=0`, then the Hessian $H_{kk}$ will be used.  If `inWrt2=1`, then
                        the second derivative will be with respect to $s$ and the Hessian $H_{ks}$ 
                        will be used.

                inputs: Just like the EvalauteImpl function, this is a list of vector-valued inputs.

                sensitivity: A vector containing the gradient of an arbitrary objective $J$
                             with respect to the output of this ModPiece.  This is the same as the 
                             sensitivity vector used in the `GradientImpl` function.
                             
                vec : The vector we want to apply the Hessian to.  The length of this vector should 
                      be the same as the length of `inputs[0]` when inWrt2=0 and should be the same 
                      length as `sensitivity` when `inWrt2=1`.

            RETURNS:
                This function returns nothing.  It stores the result in the `ApplyHessian` function.

        """
        
        condVals = inputs[0]
        
        # Build the stiffness matrix
        K = self.BuildStiffness(condVals)
        
        # Solve the forward system
        sol = spla.spsolve(K,self.rhs)

        # If we're using the Hessian $\nabla_{kk} J$
        if((inWrt1==0)&(inWrt2==0)):
                        
            # Solve the adjoint system
            adjRhs = self.BuildAdjointRhs(sensitivity)
            adjSol = spla.spsolve(K.T,adjRhs) # Because K is symmetric

            # Solve the incremental forward system
            incrForRhs = self.BuildIncrementalRhs(sol, vec)
            incrForSol = spla.spsolve(K,incrForRhs)
            
            # Solve the incremental adjoint system
            incrAdjRhs = self.BuildIncrementalRhs(adjSol,vec)
            incrAdjSol = spla.spsolve(K.T,incrAdjRhs) # Because K is symmetric
            
            # Construct the Hessian action
            solDeriv = (sol[1:]-sol[:-1])/self.dx
            adjDeriv = (adjSol[1:]-adjSol[:-1])/self.dx
            incrForDeriv = (incrForSol[1:]-incrForSol[:-1])/self.dx
            incrAdjDeriv = (incrAdjSol[1:]-incrAdjSol[:-1])/self.dx
            
            self.hessAction = (incrAdjDeriv * solDeriv + incrForDeriv * adjDeriv)#*2.
            
        # If we're using the mixed Hessian $\nabla_{ks} J$
        elif(((inWrt1==0)&(inWrt2==1))|((inWrt1==1)&(inWrt2==0))):
            
            temp = spla.spsolve(K.T, vec)
            dhdx = (sol[1:] - sol[:-1])/self.dx   # derivative of forward solution, which is constant in each cell
            dtempdx = (temp[1:] - temp[:-1])/self.dx # derivative of adjoint solution, which is constant in each cell
            
            self.hessAction = -self.dx * dhdx * dtempdx 
        
        else: 
            assert(False)

    def BuildStiffness(self, condVals):
        """ Constructs the stiffness matrix for the conductivity defined within each cell. """

        rows = []
        cols = []
        vals = []
        
       
        # Left Dirichlet BC
        rows.append(0)
        cols.append(0)
        vals.append(1e10) # Trick to approximately enforce Dirichlet BC at x=0 while keeping matrix symmetric

        # Integration over each cell
        for cellInd in range(self.numCells):
            leftNode = cellInd
            rightNode = cellInd+1
            
            rows.append(leftNode)
            cols.append(rightNode)
            vals.append(-condVals[cellInd]/self.dx)
            
            rows.append(rightNode)
            cols.append(leftNode)
            vals.append(-condVals[cellInd]/self.dx)
            
            rows.append(rightNode)
            cols.append(rightNode)
            vals.append(condVals[cellInd]/self.dx)
            
            rows.append(leftNode)
            cols.append(leftNode)
            vals.append(condVals[cellInd]/self.dx)

        return sp.csr_matrix((vals,(rows,cols)), shape=(self.numNodes, self.numNodes))
    
    def BuildRhs(self, recharge):
        """ Constructs the right hand side vector for the forward problem. """

        rhs = np.zeros((self.numNodes,))

        rhs[1:-1] = 0.5*self.dx*(recharge[0:-1] + recharge[1:])
        rhs[-1] = 0.5*self.dx*recharge[-1]
        return rhs

    def BuildAdjointRhs(self,sensitivity):
        """ Constructs the right hand side vector for the adjoint problem. """
        rhs = -1.0*sensitivity 
        rhs[0] = 0.0
        return rhs

    def BuildIncrementalRhs(self, sol, khat):
        """ Constructs the right hand side vector for both incremental problems. """
        # The derivative of the solution in each cell
        solGrad = (sol[1:]-sol[:-1])/self.dx
        rhs = np.zeros((self.numNodes,))
        
        for cellInd in range(self.numCells):
            leftNode = cellInd
            rightNode = cellInd+1
            
            rhs[leftNode] -= self.dx*khat[cellInd]*solGrad[cellInd]
            rhs[rightNode] += self.dx*khat[cellInd]*solGrad[cellInd]
            
        rhs[0]=0.0
        return -rhs
    

## Construct Model

In [3]:
numCells = 200
nodeXs = np.linspace(0,1,numCells+1) # The locations $x_i$ of each node in the discretization
cellXs = 0.5*(nodeXs[1:]+nodeXs[0:-1]) # A point at the center of each cell.

recharge = np.ones(numCells)

mod = FlowEquation(recharge)

In [4]:
mean = np.zeros(numCells+1)
objective = mm.Gaussian(mean).AsDensity()

## Evaluate Model

In [5]:
K = np.exp(1.0*np.cos(20.0*cellXs))

h = mod.Evaluate([K])[0]

NameError: name 'cellXs' is not defined

In [ ]:
fig, axs = plt.subplots(nrows=2,sharex=True,figsize=(12,6))

axs[0].step(nodeXs[0:-1],K,linewidth=2,where='post')
axs[0].set_ylabel('$K(x)$',fontsize=16)

axs[1].plot(nodeXs,h,linewidth=2)
axs[1].set_ylabel('$h(x)$',fontsize=16)
axs[1].set_xlabel('$x$', fontsize=16)
plt.subplots_adjust(hspace=0.05)
plt.show()

## Check Gradient of Model

In [ ]:
sens = objective.Gradient(0,0,[h],np.ones(1))

grad = mod.Gradient(0,0,[K],sens)
gradFD = mod.GradientByFD(0,0,[K],sens)

In [ ]:
fig, axs = plt.subplots(nrows=4,sharex=True,figsize=(12,13))

axs[0].plot(nodeXs, sens, linewidth=2)
axs[0].set_ylabel('Sensitivity $\\nabla_h J$',fontsize=16)

axs[1].step(nodeXs[0:-1], grad, where='post',linewidth=2)
axs[1].set_ylabel('Adjoint-based $\\nabla_K J$',fontsize=16)

axs[2].step(nodeXs[0:-1], gradFD, where='post',linewidth=2)
axs[2].set_ylabel('Finite Difference',fontsize=16)

axs[3].step(nodeXs[0:-1], np.abs(grad-gradFD), where='post',linewidth=2)
axs[3].set_ylabel('Absolute Difference',fontsize=16)

axs[3].set_xlabel('$x$',fontsize=16)

plt.subplots_adjust(hspace=0.1)
plt.show()


## Test Hessian of Model

In [ ]:
hessDir = np.random.rand(K.shape[0])

hessAct = mod.ApplyHessian(0,0,0,[K],sens,hessDir)
hessActFD = mod.ApplyHessianByFD(0,0,0,[K],sens,hessDir)

In [ ]:
fig, axs = plt.subplots(nrows=4, figsize=(12,13), sharex=True)

axs[0].step(nodeXs[0:-1], hessDir, where='post', linewidth=2)
axs[0].set_ylabel('Vector $v$', fontsize=16)

axs[1].step(nodeXs[0:-1], hessAct, where='post', linewidth=2)
axs[1].set_ylabel('Adjoint-based $Hv$',fontsize=16)

axs[2].step(nodeXs[0:-1], hessActFD, where='post', linewidth=2)
axs[2].set_ylabel('Finite Difference',fontsize=16)

axs[3].step(nodeXs[0:-1], np.abs(hessAct-hessActFD), where='post', linewidth=2)
axs[3].set_ylabel('Absolute Difference',fontsize=16)
axs[3].set_xlabel('$x$', fontsize=16)

plt.subplots_adjust(hspace=0.1)
plt.show()

## Test Hessian of Objective

In [ ]:
graph = mm.WorkGraph()
graph.AddNode(mod, 'Model')
graph.AddNode(objective, 'Objective')
graph.AddEdge('Model',0,'Objective',0)

fullMod = graph.CreateModPiece('Objective')

sens = np.ones(1)

hessAct = fullMod.ApplyHessian(0,0,0,[K],sens,hessDir)
hessActFD = fullMod.ApplyHessianByFD(0,0,0,[K],sens,hessDir)

In [ ]:
fig, axs = plt.subplots(nrows=4, figsize=(12,13), sharex=True)

axs[0].step(nodeXs[0:-1], hessDir, where='post', linewidth=2)
axs[0].set_ylabel('Vector $v$', fontsize=16)

axs[1].step(nodeXs[0:-1], hessAct, where='post', linewidth=2)
axs[1].set_ylabel('Adjoint-based $Hv$',fontsize=16)

axs[2].step(nodeXs[0:-1], hessActFD, where='post', linewidth=2)
axs[2].set_ylabel('Finite Difference $Hv$',fontsize=16)

axs[3].step(nodeXs[0:-1], np.abs(hessAct-hessActFD), where='post', linewidth=2)
axs[3].set_ylabel('Absolute Difference',fontsize=16)
axs[3].set_xlabel('$x$', fontsize=16)

plt.subplots_adjust(hspace=0.1)
plt.show()

## Hessian Spectrum

#### Define the linear operator

In [ ]:
outWrt = 0 # There's only one output of "fullMod", so this is the only possibility
inWrt1 = 0 # There's also only one input of "fullMod"
inWrt2 = 0

scaling = -1.0
hessOp = mm.HessianOperator(fullMod, [K], outWrt, inWrt1, inWrt2, sens, scaling)

#### Set up the eigenvalue solver

In [ ]:
opts = {
    'NumEigs' : numCells, # Maximum number of eigenvalue/eigenvector pairs to compute
    'Verbosity':3
}

solver = mm.StochasticEigenSolver(opts)
solver.compute(hessOp)

vals = solver.eigenvalues()
vecs = solver.eigenvectors()

#### Plot the eigenvalue spectrum

In [ ]:
plt.figure(figsize=(8,5))
plt.semilogy(vals,linewidth=2)
plt.ylabel('Eigenvalue $\lambda_i$',fontsize=16)
plt.xlabel('Index $i$',fontsize=16)
plt.show()